In [1]:
from src.tests import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
result = joblib.load("Experiment results/Conversion/Thompson/Experiment2/p_control=1__mde=10__prob_super=0.95__batch_size_share_mu=0.01__method_update_params=summation__multi_armed=True")


In [ ]:
get_size_zratio(1, 10, alpha=0.05, beta=0.2)

In [ ]:
pd.Series(Counter([result[i][1][0].shape[0] for i in range(1000)])).sort_values().tail(60)

# Experiment 2
## Thompson

In [2]:
test_result_mab_2 = TestResult2Processing(folder="Thompson/Experiment2/", test_type="thompson")
results_mab_2 = test_result_mab_2.run()
results_mab_2_fp = results_mab_2.query("mde == '0'")
results_mab_2_fn = results_mab_2.query("mde != '0'")
results_mab_2_fn['conversion_share_mean'] = results_mab_2_fn['conversion_sum_mean'] / (results_mab_2_fn['size'] * 2)
results_mab_2_fn['conversion_share_median'] = results_mab_2_fn['conversion_sum_median'] / (results_mab_2_fn['size'] * 2)

results_mab_2_fn.unstack(level='multi_armed').to_excel(
    "Description experiment results/Conversion/Experiment2/thompson_FN.xlsx")
results_mab_2_fp.unstack(level='multi_armed').to_excel(
    "Description experiment results/Conversion/Experiment2/thompson_FP.xlsx")

  0%|          | 0/720 [00:00<?, ?it/s]

In [9]:
results_mab_2_fn.unstack(level='multi_armed')[['share_test_winner', 
                                               'conversion_sum_mean',
                                              'conversion_sum_median']]

share_test_winner  \
multi_armed                                                                   False   
p_control mde prob_super batch_size_share_mu method_update_params                     
1         10  0.9        0.01                normalization                    0.593   
                                             summation                        0.620   
                         0.02                normalization                    0.654   
                                             summation                        0.698   
                         0.03                normalization                    0.696   
...                                                                             ...   
6         5   0.95       0.08                summation                        0.770   
                         0.09                normalization                    0.708   
                                             summation                        0.769   
                         0.1                 normalization                    0.699   
                                             summation                        0.780   

                                                                          \
multi_armed                                                         True   
p_control mde prob_super batch_size_share_mu method_update_params          
1         10  0.9        0.01                normalization         0.518   
                                             summation             0.524   
                         0.02                normalization         0.546   
                                             summation             0.576   
                         0.03                normalization         0.601   
...                                                                  ...   
6         5   0.95       0.08                summation             0.602   
                         0.09                normalization         0.595   
                                             summation             0.621   
                         0.1                 normalization         0.610   
                                             summation             0.629   

                                                                  conversion_sum_mean  \
multi_armed                                                                     False   
p_control mde prob_super batch_size_share_mu method_update_params                       
1         10  0.9        0.01                normalization                      776.5   
                                             summation                          918.2   
                         0.02                normalization                      888.8   
                                             summation                         1042.0   
                         0.03                normalization                     1040.6   
...                                                                               ...   
6         5   0.95       0.08                summation                         8158.1   
                         0.09                normalization                     5859.5   
                                             summation                         8503.3   
                         0.1                 normalization                     6075.7   
                                             summation                         8730.5   

                                                                           \
multi_armed                                                          True   
p_control mde prob_super batch_size_share_mu method_update_params           
1         10  0.9        0.01                normalization         1068.8   
                                             summation             1580.6   
                         0.02                normalization         1239.2   
                                             summation             1640.3

In [ ]:
results_mab_2_fn.sort_values("conversion_share_mean", ascending=False)[
    ['conversion_share_median', 'share_test_winner']
].astype(float).corr()

In [14]:
results_mab_2_fn.sort_values("conversion_share_mean", ascending=False).\
query("multi_armed=='False'").reset_index()[
    ['batch_size_share_mu', 'conversion_share_median', 
     'share_test_winner', 'number_of_iteration_median',
     'number_of_iteration_mean']].astype(float).corr()

,batch_size_share_mu,conversion_share_median,share_test_winner,number_of_iteration_median,number_of_iteration_mean
batch_size_share_mu,1.000000,0.380031,0.810251,-0.496096,-0.781433
conversion_share_median,0.380031,1.000000,0.119252,-0.042102,-0.268235
share_test_winner,0.810251,0.119252,1.000000,-0.789798,-0.864186
number_of_iteration_median,-0.496096,-0.042102,-0.789798,1.000000,0.821542
number_of_iteration_mean,-0.781433,-0.268235,-0.864186,0.821542,1.000000


In [13]:
results_mab_2_fn.query("multi_armed=='False'")

count_winners  \
p_control mde prob_super batch_size_share_mu method_update_params multi_armed                              
1         10  0.95       0.01                summation            False         {1: 603, -1: 97, 0: 300}   
6         5   0.9        0.09                normalization        False         {1: 754, -1: 196, 0: 50}   
11        5   0.95       0.07                summation            False         {-1: 154, 1: 759, 0: 87}   
1         5   0.95       0.03                normalization        False        {1: 616, -1: 237, 0: 147}   
11        5   0.9        0.09                normalization        False         {1: 762, 0: 65, -1: 173}   
...                                                                                                  ...   
1         10  0.95       0.04                summation            False        {1: 718, -1: 140, 0: 142}   
              0.9        0.09                summation            False         {1: 804, 0: 81, -1: 115}   
6         5   0.95       0.05                summation            False        {1: 726, 0: 119, -1: 155}   
1         5   0.95       0.02                normalization        False        {1: 596, -1: 203, 0: 201}   
6         10  0.9        0.05                summation            False        {1: 746, 0: 149, -1: 105}   

                                                                                 size  \
p_control mde prob_super batch_size_share_mu method_update_params multi_armed           
1         10  0.95       0.01                summation            False        163091   
6         5   0.9        0.09                normalization        False        100667   
11        5   0.95       0.07                summation            False         51908   
1         5   0.95       0.03                normalization        False        637007   
11        5   0.9        0.09                normalization        False         51908   
...                                                                               ...   
1         10  0.95       0.04                summation            False        163091   
              0.9        0.09                summation            False        163091   
6         5   0.95       0.05                summation            False        100667   
1         5   0.95       0.02                normalization        False        637007   
6         10  0.9        0.05                summation            False         25736   

                                                                              share_observations_mean  \
p_control mde prob_super batch_size_share_mu method_update_params multi_armed                           
1         10  0.95       0.01                summation            False                [0.415, 0.415]   
6         5   0.9        0.09                normalization        False                [0.765, 0.765]   
11        5   0.95       0.07                summation            False                [0.649, 0.649]   
1         5   0.95       0.03                normalization        False                [0.803, 0.803]   
11        5   0.9        0.09                normalization        False                [0.746, 0.746]   
...                                                                                               ...   
1         10  0.95       0.04                summation            False                [0.568, 0.568]   
              0.9        0.09                summation            False                [0.524, 0.524]   
6         5   0.95       0.05                summation            False                [0.604, 0.604]   
1         5   0.95       0.02                normalization        False                [0.711, 0.711]   
6         10  0.9        0.05                summation            False                  [0.45, 0.45]   

                                                                              share_observations_median  \
p_control mde prob_super batch_size_share_mu method_upd

In [ ]:
results_mab_2_fn.query("multi_armed=='True'").sort_values("share_test_winner", ascending=False)

In [ ]:
results_mab_2_fn.

# Experiment 3
## Thompson

In [ ]:
test_result_mab_3 = TestResult2Processing(folder="Thompson/Experiment3/", test_type="thompson")
results_mab_3 = test_result_mab_3.run()
results_mab_3_fp = results_mab_3.query("mde == '0'")
results_mab_3_fn = results_mab_3.query("mde != '0'")

In [ ]:
results_mab_3_fn.sort_values("share_test_winner", ascending=False)

# Compare experiments 

## Thompson

In [ ]:
results_mab_3_fp[['share_not_winner']].join(
results_mab_2_fp[['share_not_winner']], rsuffix='_exp2', lsuffix='_exp3')

In [ ]:
results_mab_3_fn[['share_test_winner']].join(
results_mab_2_fn[['share_test_winner']], rsuffix='_exp2', lsuffix='_exp3').\
sort_values("share_test_winner_exp3", ascending=False).iloc[60:120]

In [ ]:
# test_result_mab = TestResult1Processing(folder="Thompson/Experiment1/",
#                                         test_type="thompson")
# test_result_bayes = TestResult1Processing(folder="Bayesian/Experiment1/",
#                                           test_type="bayesian")
# test_result_mab = TestResult1Processing(folder="Thompson/Experiment2/",
#                                         test_type="thompson")
# test_result2_bayes = TestResult2Processing(folder="Bayesian/Experiment2/",
#                                            test_type="bayesian")

# results_bayes = test_result_bayes.run()
# results_bayes_fp = results_bayes.query("mde == '0'")

# # # results_bayes_fn = results_bayes.query("mde != '0'")
# results2_bayes = test_result2_bayes.run()
# results2_bayes_fp = results2_bayes.query("mde == '0'")
# results2_bayes_fn = results2_bayes.query("mde != '0'")
# results2_bayes_fp.reset_index()[
#     ['share_observation_optimal_arms', 'share_not_winner']
# ].astype(float).corr()
# n_samples = results2_bayes_fn[['share_test_winner']].unstack()
# n_samples_fp = results2_bayes_fp[['share_not_winner']].unstack()



In [ ]:
thompson_results_df.query("mde == '0'")['share_not_winner'].plot(kind='hist');
plt.title("Доля тестов, которые зафиксировали, что различий нет, когда их на самом деле нет");

In [ ]:
plot_df = thompson_results_df.query("""mde == '0' & multi_armed == 'True'""").reset_index()[
    ['share_not_winner', "prob_super",'batch_size_share_mu']].astype(float)
fig = px.scatter(plot_df, x="share_not_winner", y="prob_super", color="batch_size_share_mu")
fig.show()

In [ ]:
plot_df = thompson_results_df.query("""mde != '0' & multi_armed == 'False'""").reset_index()[
    ["share_not_winner", 'share_test_winner', "prob_super",'batch_size_share_mu']].astype(float)
fig = px.scatter(plot_df, x="share_test_winner", y="prob_super", color="batch_size_share_mu")
fig.show()